# 不用正则表达式来查找文本模式

假设你希望在字符串中查找电话号码。你知道模式：3 个数字，一个短横线，3个数字，一个短横线，再是4 个数字。例如：415-555-4242。

In [3]:
def isPhoneNumber(text):
    if len(text)!= 12:
        return False
    if not text[:3].isdecimal() :
        return False
    if text[3]!= '-':
        return False
    if not text[4:7].isdecimal():
        return False
    if text[7]!= '-':
        return False
    if not text[8:].isdecimal():
        return False
    return True

print(isPhoneNumber('415-555-4242'))

True


必须添加更多代码，才能在更长的字符串中寻找这种文本模式。用下面的代码，
替代isPhoneNumber.py 中最后4 个print()函数调用：

In [4]:
message = 'Call me at 415-555-1011 tomorrow. 415-555-9999 is my office.'
for i in range(len(message)):
    chunk = message[i:i+12]
    if isPhoneNumber(chunk):
        print('Phone number found: ' + chunk)
print('Done')

Phone number found: 415-555-1011
Phone number found: 415-555-9999
Done


# 用正则表达式查找文本模式
正则表达式，简称为regex，是文本模式的描述方法。**例如，\d 是一个正则表
达式，表示一位数字字符，即任何一位 0 到 9 的数字**。Python 使用正则表达式
\d\d\d-\d\d\d-\d\d\d\d，来匹配前面isPhoneNumber()函数匹配的同样文本：3 个数字、
一个短横线、3 个数字、一个短横线、4 个数字。所有其他字符串都不能匹配
\d\d\d-\d\d\d-\d\d\d\d 正则表达式。

但正则表达式可以复杂得多。例如，在一个模式后加上花括号包围的3（{3}），
就是说，“匹配这个模式3 次”。所以较短的正则表达式\d{3}-\d{3}-\d{4}，也匹配正
确的电话号码格式。

## 创建正则表达式对象
向`re.compile()`传入一个字符串值，表示正则表达式，它将返回一个Regex 模式
对象（或者就简称为Regex 对象）。创建一个Regex 对象来匹配电话号码模式:

In [1]:
import re
phoneNumberRegex = re.compile(r'\d\d\d-\d\d\d-\d\d\d\d')

## 匹配Regex对象

In [6]:
mo = phoneNumberRegex.search('Call me at 415-555-1011 tomorrow.')
mo.group() 

'415-555-1011'

# 用正则表达式匹配更多模式

## 利用括号分组
假定想要将区号从电话号码中分离。添加括号将在正则表达式中创建“分组”：
(\d\d\d)-(\d\d\d-\d\d\d\d)。然后可以使用group()匹配对象方法，从一个分组中获取匹
配的文本。

正则表达式字符串中的第一对括号是第1 组。第二对括号是第2 组。向group()
匹配对象方法传入整数1 或2，就可以取得匹配文本的不同部分。向group()方法传
入0 或不传入参数，将返回整个匹配的文本。

In [7]:
phoneNumRegex = re.compile(r'(\d\d\d)-(\d\d\d-\d\d\d\d)')
mo = phoneNumRegex.search('My number is 415-555-4242.')
mo.group(1)

'415'

In [11]:
mo.group(0)

'415-555-4242'

In [12]:
# 一次就获取所有的分组
mo.groups()

('415', '555-4242')

括号在正则表达式中有特殊的含义，但是如果你需要在文本中匹配括号，怎么
办？例如，你要匹配的电话号码，可能将区号放在一对括号中。在这种情况下，就
需要用倒斜杠对(和)进行字符转义。

In [15]:
phoneNumRegex = re.compile(r'(\(\d\d\d\)) (\d\d\d-\d\d\d\d)')
mo = phoneNumRegex.search('My phone number is (415) 555-4242.')
mo.group(1)

'(415)'

## 用管道匹配多个分组
字符`|`称为“**管道**”。希望匹配许多表达式中的一个时，就可以使用它。例如，
正则表达式`r'Batman|Tina Fey'`将匹配'Batman'或'Tina Fey'。

In [16]:
heroRegex = re.compile (r'Batman|Tina Fey')
mo1 = heroRegex.search('Batman and Tina Fey.')
mo1.group() # 利用findall()方法，可以找到“所有”匹配的地方。

'Batman'

In [17]:
mo2 = heroRegex.search('Tina Fey and Batman.')
mo2.group()

'Tina Fey'

也可以使用管道来匹配多个模式中的一个，作为正则表达式的一部分。假设你希望匹配'Batman'、'Batmobile'、'Batcopter'和'Batbat'中任意一个。因为所有这
些字符串都以Bat 开始，所以如果能够只指定一次前缀非常方便，可以通过括
号实现。

In [18]:
batRegex = re.compile(r'Bat(man|mobile|copter|bat)')
mo = batRegex.search('Batmobile lost a wheel')
mo.group()

'Batmobile'

In [20]:
mo.group(1)

'mobile'

方法调用mo.group()返回了完全匹配的文本'Batmobile'，而mo.group(1)只是返
回第一个括号分组内匹配的文本'mobile'。通过使用管道字符和分组括号，可以指定
几种可选的模式，让正则表达式去匹配。

## 用问号实现可选匹配
想匹配的模式是可选的。就是说，不论这段文本在不在，正则表达式
都会认为匹配。**字符?表明它前面的分组在这个模式中是可选的。**

In [21]:
batRegex = re.compile(r'Bat(wo)?man')
mo1 = batRegex.search('The Adventures of Batman')
mo1.group()

'Batman'

In [22]:
mo2 = batRegex.search('The Adventures of Batwoman')
mo2.group()

'Batwoman'

In [24]:
mo2.groups()

('wo',)

## 用星号匹配零次或多次
\*（称为星号）意味着“匹配零次或多次”，即星号之前的分组，可以在文本中出现任意次。。它可以完全不存在，或一次又一次地重复。

In [28]:
batRegex = re.compile(r'Bat(wo)*man')
mo1 = batRegex.search('The Adventures of Batman')
mo1.group()

'Batman'

In [29]:
mo3 = batRegex.search('The Adventures of Batwowowowoman')
mo3.group()

'Batwowowowoman'

In [30]:
mo3.groups()

('wo',)

## 用加号匹配一次或多次
星号不要求分组出现在匹配的字符串中，但加号不同，加号前面的分组必须“至少出现一次”。这不
是可选的。

In [32]:
batRegex = re.compile(r'Bat(wo)+man')
mo1 = batRegex.search('The Adventures of Batwoman')
mo1.group()

'Batwoman'

In [33]:
mo2 = batRegex.search('The Adventures of Batwowowowoman')
mo2.group()

'Batwowowowoman'

In [34]:
# 正则表达式Bat(wo)+man 不会匹配字符串，因为加号要求wo 至少出现一次。
mo3 = batRegex.search('The Adventures of Batman')
mo3 == None

True

## 用花括号匹配特定次数
如果想要一个分组重复特定次数，就在正则表达式中该分组的后面，跟上花括
号包围的数字。例如，正则表达式(Ha){3}将匹配字符串'HaHaHa'，但不会匹配'HaHa'，
因为后者只重复了(Ha)分组两次。

除了一个数字，还可以指定一个范围，即在花括号中写下一个最小值、一个逗号和
一个最大值。例如，正则表达式(Ha){3,5}将匹配'HaHaHa'、'HaHaHaHa'和'HaHaHaHaHa'。

也可以不写花括号中的第一个或第二个数字，不限定最小值或最大值。(Ha){3,}将匹配3 次或更多次实例，(Ha){,5}将匹配0 到5 次实例。花括号让正则表
达式更简短。

In [35]:
haRegex = re.compile(r'(Ha){3}')
mo1 = haRegex.search('HaHaHa')
mo1.group()

'HaHaHa'

In [36]:
mo2 = haRegex.search('HaHa')
mo2 == None

True

# 贪心和非贪心匹配
在字符串'HaHaHaHaHa'中，因为(Ha){3,5}可以匹配3 个、4 个或5 个实例，你可能
会想，为什么在前面花括号的例子中，Match 对象的group()调用会返回'HaHaHaHaHa'，
而不是更短的可能结果。毕竟，'HaHaHa'和'HaHaHaHa'也能够有效地匹配正则表达
式(Ha){3,5}。

Python 的正则表达式默认是“**贪心**”的，这表示**在有二义的情况下，它们会尽
可能匹配最长的字符串**。花括号的“**非贪心**”版本**匹配尽可能最短的字符串**，即在
结束的花括号后跟着一个问号。

In [37]:
greedyRegex = re.compile(r'(Ha){3,5}')
mo1 = greedyRegex.search('HaHaHaHaHa')
mo1.group()

'HaHaHaHaHa'

In [38]:
nongreedyHaRegex = re.compile(r'(Ha){3,5}?')
mo2 = nongreedyHaRegex.search('HaHaHaHaHa')
mo2.group()

'HaHaHa'

请注意，问号在正则表达式中可能有两种含义：声明非贪心匹配或表示可选的
分组。这两种含义是完全无关的。

# findall()方法
除了search 方法外，Regex 对象也有一个findall()方法。search()将返回一个Match
对象，包含被查找字符串中的“第一次”匹配的文本，而findall()方法将返回一组
字符串，包含被查找字符串中的所有匹配。search()返回的Match 对象只
包含第一次出现的匹配文本。

In [39]:
phoneNumRegex = re.compile(r'\d\d\d-\d\d\d-\d\d\d\d')
mo = phoneNumRegex.search('Cell: 415-555-9999 Work: 212-555-0000')
mo.group()

'415-555-9999'

findall()不是返回一个Match 对象，而是返回一个字符串列表，只要
在正则表达式中没有分组。列表中的每个字符串都是一段被查找的文本，它匹配该
正则表达式。

In [40]:
phoneNumRegex = re.compile(r'\d\d\d-\d\d\d-\d\d\d\d')
phoneNumRegex.findall('Cell: 415-555-9999 Work: 212-555-0000')

['415-555-9999', '212-555-0000']

如果在正则表达式中有分组，那么findall 将返回元组的列表。每个元组表示一个找
到的匹配，其中的项就是正则表达式中每个分组的匹配字符串

In [42]:
phoneNumRegex = re.compile(r'(\d\d\d)-(\d\d\d)-(\d\d\d\d)') # has groups
phoneNumRegex.findall('Cell: 415-555-9999 Work: 212-555-0000')

[('415', '555', '9999'), ('212', '555', '0000')]

# 字符分类
在前面电话号码正则表达式的例子中，你知道\d 可以代表任何数字。也就是说，\d
是正则表达式(0|1|2|3|4|5|6|7|8|9)的缩写。有许多这样的“缩写字符分类”



| 编写字符分类 | 表示                                                   |
| ------------ | ------------------------------------------------------ |
| \d           | 0到9的任何数字                                         |
| \D           | 除0 到9 的数字以外的任何字符                           |
| \w           | 任何字母、数字或下划线字符（可以认为是匹配“单词”字符） |
| \W           | 除字母、数字和下划线以外的任何字符                     |
| \s           | 空格、制表符或换行符（可以认为是匹配“空白”字符）       |
| \S           | 除空格、制表符和换行符以外的任何字符                   |



In [49]:
xmasRegex = re.compile(r'\d+\s\w+')
xmasRegex.findall('12 drummers, 11 pipers, 10 lords, 9 ladies, 8 maids, 7 swans, 6 geese, 5 rings, 4 birds, 3 hens, 2 doves, 1 partridge')

['12 drummers',
 '11 pipers',
 '10 lords',
 '9 ladies',
 '8 maids',
 '7 swans',
 '6 geese',
 '5 rings',
 '4 birds',
 '3 hens',
 '2 doves',
 '1 partridge']

# 建立自己的字符分类
有时候你想匹配一组字符，但缩写的字符分类（\d、\w、\s 等）太宽泛。你可
以用**方括号**定义自己的字符分类。例如，字符分类[aeiouAEIOU]将匹配所有元音字
符，不论大小写。

In [50]:
vowelRegex = re.compile(r'[aeiouAEIOU]')
vowelRegex.findall('RoboCop eats baby food. BABY FOOD.')

['o', 'o', 'o', 'e', 'a', 'a', 'o', 'o', 'A', 'O', 'O']

也可以使用短横表示字母或数字的范围。例如，字符分类[a-zA-Z0-9]将匹配所
有小写字母、大写字母和数字。

请注意，在方括号内，普通的正则表达式符号不会被解释。这意味着，你不需
要前面加上倒斜杠转义.、*、?或()字符。例如，字符分类将匹配数字0 到5 和一个
句点。你不需要将它写成[0-5\.]。

通过在字符分类的**左方括号后加上一个插入字符（^）**，就可以得到“**非字符类**”。
非字符类将匹配不在这个字符类中的所有字符。

In [51]:
consonantRegex = re.compile(r'[^aeiouAEIOU]')
consonantRegex.findall('RoboCop eats baby food. BABY FOOD.')

['R',
 'b',
 'C',
 'p',
 ' ',
 't',
 's',
 ' ',
 'b',
 'b',
 'y',
 ' ',
 'f',
 'd',
 '.',
 ' ',
 'B',
 'B',
 'Y',
 ' ',
 'F',
 'D',
 '.']

# 插入字符和美元字符
可以在正则表达式的开始处使用插入符号（`^`），表明匹配必须发生在被查找文本开始处。类似地，可以再正则表达式的末尾加上美元符号（`$`），表示该字符串必须以这个正则表达式的模式结束。

可以同时使用`^`和`$`，表明整个字符串必须匹配该模式，也就是说，只匹配该字符串的某个子集是不够的。

In [56]:
# '匹配以'Hello'开始的字符串
beginsWithHello = re.compile(r'^Hello')
beginsWithHello.search('Hello World!')

<_sre.SRE_Match object; span=(0, 5), match='Hello'>

In [57]:
beginsWithHello.search('He said hello.') == None

True

正则表达式r'\d$'匹配以数字0 到9 结束的字符串。

In [59]:
endWithNumber = re.compile(r'\d$')
endWithNumber.search('your number is 42')

<_sre.SRE_Match object; span=(16, 17), match='2'>

In [60]:
endWithNumber.search('Your number is forty two.') == None

True

正则表达式r'^\d+$'匹配从开始到结束都是数字的字符串。

In [61]:
wholeStringIsNum = re.compile(r'^\d+$')
wholeStringIsNum.search('1234567890')

<_sre.SRE_Match object; span=(0, 10), match='1234567890'>

In [62]:
wholeStringIsNum.search('12345xyz67890') == None

True

# 通配字符
在正则表达式中，.（句点）字符称为“通配符”。它匹配除了换行之外的所有字符。

In [63]:
atRegex = re.compile(r'.at')
atRegex.findall('The cat in the hat sat on the flat mat.')

['cat', 'hat', 'sat', 'lat', 'mat']

## 用点-星匹配所有字符
有时候想要匹配所有字符串。例如，假定想要匹配字符串'First Name:'，接下来
是任意文本，接下来是'Last Name:'，然后又是任意文本。可以用点-星（.*）表示“任
意文本”。回忆一下，句点字符表示“除换行外所有单个字符”，星号字符表示“前
面字符出现零次或多次”。

In [64]:
nameRegex = re.compile(r'First Name: (.*) Last Name: (.*)')
mo = nameRegex.search('First Name: Al Last Name: Sweigart')
mo.groups()

('Al', 'Sweigart')

点-星使用“贪心”模式：它总是匹配尽可能多的文本。要用“非贪心”模式匹配
所有文本，就使用点-星和问号。像和大括号一起使用时那样，问号告诉Python 用非贪
心模式匹配。在

In [75]:
nongreedyRegex = re.compile(r'<.*?>')
mo = nongreedyRegex.search('<To serve man> for dinner.>')
mo.group()

'<To serve man>'

In [76]:
greedyRegex = re.compile(r'<.*>')
mo = greedyRegex.search('<To serve man> for dinner.>')
mo.group()

'<To serve man> for dinner.>'

## 用句点字符匹配换行

点-星将匹配除换行外的所有字符。通过传入re.DOTALL 作为re.compile()的第
二个参数，可以让句点字符匹配所有字符，包括换行字符。

In [77]:
noNewlineRegex = re.compile('.*')
noNewlineRegex.search('Serve the public trust.\nProtect the innocent.\nUphold the law.').group()

'Serve the public trust.'

In [78]:
newlineRegex = re.compile('.*', re.DOTALL)
newlineRegex.search('Serve the public trust.\nProtect the innocent.\nUphold the law.').group()

'Serve the public trust.\nProtect the innocent.\nUphold the law.'

正则表达式noNewlineRegex 在创建时没有向re.compile()传入re.DOTALL，它
将匹配所有字符，直到第一个换行字符。但是，newlineRegex 在创建时向re.compile()传
入了re.DOTALL，它将匹配所有字符。这就是为什么newlineRegex.search()调用匹配完
整的字符串，包括其中的换行字符。

# 复习
- ?匹配零次或一次前面的分组。
- *匹配零次或多次前面的分组。
- +匹配一次或多次前面的分组。
- {n}匹配n 次前面的分组。
- {n,}匹配n 次或更多前面的分组。
- {,m}匹配零次到m 次前面的分组。
- {n,m}匹配至少n 次、至多m 次前面的分组。
- {n,m}?或*?或+?对前面的分组进行非贪心匹配。
- ^spam 意味着字符串必须以spam 开始。
- spam$意味着字符串必须以spam 结束。
- .匹配所有字符，换行符除外。
- \d、\w 和\s 分别匹配数字、单词和空格。
- \D、\W 和\S 分别匹配出数字、单词和空格外的所有字符。
- [abc]匹配方括号内的任意字符（诸如a、b 或c）。
- [^abc]匹配不在方括号内的任意字符。

# 不区分大小写的匹配
有时候你只关心匹配字母，不关心它们是大写或小写。要让正则表达式
不区分大小写，可以向re.compile()传入re.IGNORECASE 或re.I，作为第二个参数。

In [79]:
robocop = re.compile(r'robocop', re.I)
robocop.search('RoboCop is part man, part machine, all cop.').group()

'RoboCop'

In [80]:
robocop.search('ROBOCOP protects the innocent.').group()

'ROBOCOP'

In [81]:
robocop.search('Al, why does your programming book talk about robocop so much?').group()

'robocop'

# 用sub()方法替换字符串

正则表达式不仅能找到文本模式，而且能够用新的文本替换掉这些模式。Regex
对象的sub()方法需要传入两个参数。

第一个参数是一个字符串，用于取代发现的匹配。第二个参数是一个待替换的字符串。sub()方法返回替换完成后的字符串。

In [83]:
nameRegex = re.compile(r'Agent \w+')
nameRegex.sub('CENSORED', 'Agent Alice gave the secret documents to Agent Bob.')

'CENSORED gave the secret documents to CENSORED.'

有时候，你可能需要使用匹配的文本本身，作为替换的一部分。在sub()的第一
个参数中，可以输入\1、\2、\3……。表示“在替换中输入分组1、2、3……的文本”。
例如，假定想要隐去密探的姓名，只显示他们姓名的第一个字母。要做到这一
点，可以使用正则表达式Agent (\w)\w*，传入r'\1****'作为sub()的第一个参数。字
符串中的\1 将由分组1 匹配的文本所替代，也就是正则表达式的(\w)分组。

In [86]:
agentNamesRegex = re.compile(r'Agent (\w)\w*')
agentNamesRegex.sub(r'\1****', 'Agent Alice told Agent Carol that Agent Eve knew Agent Bob was a double agent.')

'A**** told C**** that E**** knew B**** was a double agent.'

# 管理复杂的正则表达式
如果要匹配的文本模式很简单，正则表达式就很好。但匹配复杂的文本模式，
可能需要长的、费解的正则表达式。你可以告诉`re.compile()`，**忽略正则表达式字符
串中的空白符和注释**，从而缓解这一点。要实现这种详细模式，可以向re.compile()
传入变量`re.VERBOSE`，作为第二个参数。

你可以将正则表达式放在多行中，并加上注释，像这样：

In [87]:
phoneRegex = re.compile(r'''(
    (\d{3}|\(\d{3}\))?  # area code
    (\s|-|\.)?          # separator
    \d{3}               # first 3 digits
    (\s|-|\.)           # separator
    \d{4}               # last 4 digits
    (\s*(ext|x|ext.)\s*\d{2,5})? # extension
    )''', re.VERBOSE)

请注意，前面的例子使用了三重引号('")，创建了一个多行字符串。这样就可以
将正则表达式定义放在多行中，让它更可读。

正则表达式字符串中的注释规则，与普通的Python 代码一样：#符号和它后面直
到行末的内容，都被忽略。而且，表示正则表达式的多行字符串中，多余的空白字符
也不认为是要匹配的文本模式的一部分。这让你能够组织正则表达式，让它更可读。